In [1]:
# imports
import gmaps
import json
import numpy as np
import pandas as pd
import ipywidgets as widgets
from leuvenair.myutils.utility import readJson, getSensorData, interpolate1D, getSensorInterpolatedData 
from leuvenair.myutils.gmap_utils import get_gmap_figure
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# read all the leuvenair sensor latitudes and longitudes
# Note: fields which are not available are specified -9999
sensor_la = readJson('./leuvenair/LEUVENAIRmeta_final.json')
LAT_la = sensor_la['LAT']
LON_la = sensor_la['LON']
print('All keys in the leuvenair json file are: ',sensor_la.keys())

Note: sensors 8799 and 8827 are repeated in the json file!
All keys in the leuvenair json file are:  dict_keys(['SDS011ID', 'DHTID', 'EXPORT', 'LAT', 'LON', 'STREET', 'POSTALCODE', 'CITY', 'HEIGHT', 'SENSOR_POSITION', 'INDUSTRY', 'WOODSTOVES', 'TRAFFIC', 'SVL', 'NO2_CN'])


In [3]:
# Get all the field names in the leuvenair datadump
varname = list(pd.read_csv('./leuvenair/LEUVENAIRfulldump2018.csv', skiprows=0, nrows = 0, usecols = None).columns)
print('All variable names in the leuvenair datadump are :', varname)

All variable names in the leuvenair datadump are : ['DATEUTC', 'LAT', 'LON', 'SDS011ID', 'PM2.5', 'PM10', 'DHTID', 'TEMPERATURE', 'HUMIDITY', 'OBSID']


In [4]:
# This is going to take some time ~1-5 min
# read leuvenair sensor datadump
fields_la = getSensorData('./leuvenair/LEUVENAIRmeta_final.json','./leuvenair/LEUVENAIRfulldump2018.csv')

Note: sensors 8799 and 8827 are repeated in the json file!
The complete pandas frame has shape  (13889081, 10)
sensor: 6561  1st obs: 2018-01-30 00:00:27  last: 2018-12-31 23:59:06 median dt:  2  min, total obs: 234793
sensor: 8745  1st obs: 2018-01-30 00:01:56  last: 2018-12-31 23:58:33 median dt:  2  min, total obs: 142288
sensor: 8765  1st obs: 2018-01-30 00:00:34  last: 2018-12-31 23:58:43 median dt:  2  min, total obs: 194023
sensor: 8769  1st obs: 2018-04-22 11:15:17  last: 2018-12-31 23:58:11 median dt:  2  min, total obs: 144499
sensor: 8773  1st obs: 2018-04-05 18:45:47  last: 2018-12-24 02:28:28 median dt:  2  min, total obs: 135392
sensor: 8775  1st obs: 2018-02-01 21:41:57  last: 2018-06-01 06:03:30 median dt:  2  min, total obs: 61982
sensor: 8777  1st obs: 2018-02-02 00:02:09  last: 2018-12-31 23:57:43 median dt:  2  min, total obs: 191768
sensor: 8779 did not record any observation.
sensor: 8781  1st obs: 2018-02-01 21:42:44  last: 2018-12-31 23:58:11 median dt:  2  min,

sensor: 9631  1st obs: 2018-02-09 11:01:36  last: 2018-06-28 17:37:13 median dt:  2  min, total obs: 59178
sensor: 9633  1st obs: 2018-02-10 15:19:33  last: 2018-12-31 23:59:42 median dt:  2  min, total obs: 164500
sensor: 9635  1st obs: 2018-02-09 08:21:39  last: 2018-12-31 23:59:56 median dt:  2  min, total obs: 188724
sensor: 9637  1st obs: 2018-02-08 09:08:25  last: 2018-12-31 23:57:58 median dt:  2  min, total obs: 173580
sensor: 9639  1st obs: 2018-02-08 15:07:31  last: 2018-12-20 16:47:04 median dt:  2  min, total obs: 158470
sensor: 9643  1st obs: 2018-02-20 22:20:20  last: 2018-12-31 23:59:22 median dt:  2  min, total obs: 174476
sensor: 9753  1st obs: 2018-02-12 16:32:22  last: 2018-12-31 23:58:53 median dt:  2  min, total obs: 182917
sensor: 9801  1st obs: 2018-02-11 19:44:55  last: 2018-12-31 23:58:25 median dt:  2  min, total obs: 182042
sensor: 9812  1st obs: 2018-02-11 19:14:34  last: 2018-12-31 23:57:49 median dt:  2  min, total obs: 161731
sensor: 9825  1st obs: 2018-0

In [5]:
print('All sensors are: ',fields_la.keys())

All sensors are:  dict_keys(['6561', '8745', '8765', '8769', '8773', '8775', '8777', '8779', '8781', '8783', '8785', '8787', '8789', '8791', '8793', '8795', '8797', '8799', '8801', '8803', '8805', '8807', '8809', '8811', '8813', '8815', '8817', '8819', '8821', '8823', '8825', '8827', '8829', '8831', '8833', '8835', '8837', '8839', '8841', '8843', '8845', '8849', '8853', '8855', '8857', '8859', '8950', '8993', '9555', '9559', '9561', '9563', '9565', '9567', '9569', '9571', '9577', '9579', '9583', '9585', '9587', '9589', '9591', '9593', '9595', '9597', '9599', '9601', '9603', '9605', '9609', '9611', '9613', '9619', '9621', '9623', '9625', '9629', '9631', '9633', '9635', '9637', '9639', '9643', '9753', '9801', '9812', '9825', '10045', '10739', '10741', '11201', '11529', '12030', '12183', '12505', '12554', '12609', '12855', '13526', '13528', '13649', '15986', '16358', '16551', '16809', '19832', '19850', '21569', '22953', '24241', '24719', '26686', '30627'])


In [6]:
# extract time series data (1 minute resolution) for each sensor
# X is of shape 1 X #times
# Y is of shape #sensors X #times
X1, Y1 = getSensorInterpolatedData(fields_la,tstart='2018-04-01 00:00:00',tstop='2018-04-02 00:00:00',fid=4)
weights1=np.nanmean(Y1,axis=1)

Extracting data over duration  1 days 00:00:00  starting at  2018-04-01 00:00:00
Sampling resolution =  1  minute


In [7]:
# add layer on top of the base map
fobj1 = get_gmap_figure(LAT_la,LON_la,'./leuvenair/apikey.txt')
locations = list(zip(LAT_la,LON_la)) # provide the latitudes and longitudes
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights1)
heatmap_layer.point_radius = 0.005
heatmap_layer.dissipating = False
fobj1.add_layer(heatmap_layer)
fobj1

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
# extract time series data (1 minute resolution) for each sensor
# X is of shape 1 X #times
# Y is of shape #sensors X #times
X2, Y2 = getSensorInterpolatedData(fields_la,tstart='2018-04-03 00:00:00',tstop='2018-04-04 00:00:00',fid=4)
weights2=np.nanmean(Y2,axis=1)

Extracting data over duration  1 days 00:00:00  starting at  2018-04-03 00:00:00
Sampling resolution =  1  minute


In [9]:
# add layer on top of the base map
fobj2 = get_gmap_figure(LAT_la,LON_la,'./leuvenair/apikey.txt')
locations = list(zip(LAT_la,LON_la)) # provide the latitudes and longitudes
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights2)
heatmap_layer.point_radius = 0.005
heatmap_layer.dissipating = False
fobj2.add_layer(heatmap_layer)
fobj2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
lhs = gmaps.figure(layout={'width': '45%', 'margin': 'auto'})
rhs = gmaps.figure(center=(40.75, -74.00), zoom_level=12, layout={'width': '45%', 'margin': 'auto'})
title = widgets.HTML('<h3>Heatmaps!</h3>')
widgets.VBox([
    title,
    widgets.HBox([fobj1, fobj2], layout={'width': '100%'})
])

In [ ]:
# Problem: The left heat map is of easter evening (with high weights1),
# while right heat map is a normal day (with low weights2)
# Why do they show a very similar heat map?. I expected to see high intensity for the left heatmap.